<a href="https://colab.research.google.com/github/moey920/Python_Practice/blob/master/%ED%86%A0%EC%9D%B4%EC%8A%A4%ED%86%A0%EB%A6%AC3_%EB%8C%80%EB%B3%B8_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 토이스토리 3 대본 데이터 분석

## PDF 파일(대본) 읽어 전처리하기

In [3]:
!pip install pdfminer

     |████████████████████████████████| 4.2MB 6.4MB/s 
     |████████████████████████████████| 1.9MB 34.0MB/s 
  Created wheel for pdfminer: filename=pdfminer-20191125-cp37-none-any.whl size=6140062 sha256=c3ac8b3d2ade5f34fb3a61496ced795f3f41ea5c47212f9f1fa96fec2167858f
  Stored in directory: /root/.cache/pip/wheels/e1/00/af/720a55d74ba3615bb4709a3ded6dd71dc5370a586a0ff6f326
Successfully built pdfminer


### 일반 텍스트 추출기 코드

local에서 해당 코드를 개발할 때는 pdfminer와 관련된 모든 코드를 data_func.py로 저장하고 PyPDF2를 사용할 때 import해주세요.

In [ ]:
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(pdf_file_path):
    """
    pdf_file_path: 'dir/aaa.pdf'로 구성된 path로부터 
    내부의 text 파일을 모두 읽어서 스트링을 리턴함.
    https://pdfminersix.readthedocs.io/en/latest/tutorials/composable.html
    """
    output_string = StringIO()
    with open(pdf_file_path, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return output_string.getvalue()


toy_txt = convert_pdf_to_string("/content/drive/MyDrive/script-toystory3.pdf")
# print(toy_txt)
# print(type(convert_pdf_to_string("script-toystory3.pdf")))

In [11]:
print(type(toy_txt))

<class 'str'>


#### 차후 다른 movie scripts에서도 잘 작동하도록 할 때 필요한 함수(아직 미사용)

convert_title_to_filename: 목차에 나타나는 제목을 가져 와서 파일 이름으로 변환하는 함수-작업을 시작했을 때 더 많은 조정이 필요하다고 생각했습니다.   

split_to_title_and_pagenum:이 함수의 목적은 나중에 더 명확해질 것입니다. 기본적으로 목차의 행이 실제로 제목 (페이지 번호 쌍)인지 확인하는 데 사용되며, 그렇다면 튜플에서 반환합니다.

In [12]:
def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename


def split_to_title_and_pagenum(table_of_contents_entry):
    title_and_pagenum = table_of_contents_entry.strip()
    
    title = None
    pagenum = None
    
    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -2
            while title_and_pagenum[i].isdigit():
                i -= 1

            title = title_and_pagenum[:i].strip()
            pagenum = int(title_and_pagenum[i:].strip())
        
    return title, pagenum

The next step is about setting up the environment, we import the libraries (including the functions from the block above), check some properties of the document. And most importantly, we set up the PyPDF2 PdfFileReader object we are going to use throughout the project: reader.

In [18]:
!pip install PyPDF2
!pip install PyML

ERROR: Could not find a version that satisfies the requirement PyML (from versions: none)
ERROR: No matching distribution found for PyML


In [19]:
import PyPDF2
import csv

reader = PyPDF2.PdfFileReader(
    '/content/drive/MyDrive/script-toystory3.pdf')

print(reader.documentInfo)

num_of_pages = reader.numPages
print('Number of pages: ' + str(num_of_pages))

{'/Title': 'TS3 - Final, 11-1-10.fdx', '/Author': 'Michael Arndt', '/Creator': 'Final Draft', '/Producer': 'Mac OS X 10.5.8 Quartz PDFContext', '/CreationDate': "D:20101102003511Z00'00'", '/ModDate': "D:20101102003511Z00'00'"}
Number of pages: 131


Now we can start working with the file. Having a look at the pdf, it seems like the best course of action is to somehow extract the page numbers from the table of contents, and then use them to split the file. The table of contents is on page 3 and 4 in the pdf, which means 2 and 3 in the PdfFileReader list of PageObjects. Once we have the pdf in a separate file, we can use the pdfminer.six code to extract the text information. (Note: we could also just adjust the relevant pages directly without splitting the file, but I wanted to also create the individual pdf files, and it made sense to have a separate table of contents file too.)

이제 파일 작업을 시작할 수 있습니다. pdf를 보면 가장 좋은 방법은 목차에서 페이지 번호를 추출한 다음이를 사용하여 파일을 분할하는 것입니다. 목차는 pdf의 3 페이지와 4 페이지에 있으며 이는 PageObjects의 PdfFileReader 목록에서 2와 3을 의미합니다. 별도의 파일에 pdf가 있으면 pdfminer.six 코드를 사용하여 텍스트 정보를 추출 할 수 있습니다. (참고 : 파일을 분할하지 않고 관련 페이지를 직접 조정할 수도 있지만 개별 pdf 파일도 만들고 싶었고 별도의 목차 파일도있는 것이 합리적이었습니다.)

In [28]:
writer = PyPDF2.PdfFileWriter()

for page in range(2,4):

    writer.addPage(reader.getPage(page))
    
output_filename = '/content/drive/MyDrive/table_of_contents.pdf'

with open(output_filename, 'wb') as output:
    writer.write(output)

# 로컬에서 개발할 경우 import한 data_func.convert_pdf_to_string로 쓰면 됩니다.
text = convert_pdf_to_string(
    '/content/drive/MyDrive/table_of_contents.pdf')

In [37]:
print(text[:200])

GALLOPING HOOVES
TILT UP to find Woody, riding BULLSEYE, JESSIE at the reins.

              2

JESSIE

I believe you dropped something, 
Mister!

Jessie?!

ONE-EYED BART 

WOODY

You’ve reached the e


In [38]:
text = text.replace('.','')
text = text.replace('\x0c','')
table_of_contents_raw = text.split('\n')

We would like to collect the titles and page numbers into lists. This is done by stripping away the space characters, and then checking if the line ends with numbers. We need to be careful because we do want to keep the spaces within the titles.


제목과 페이지 번호를 목록으로 모으고 싶습니다. 이것은 공백 문자를 제거한 다음 행이 숫자로 끝나는 지 확인하여 수행됩니다. 제목 내에 공백을 유지하고 싶기 때문에주의해야합니다.

In [40]:
title_list = []
pagenum_list = []
title_formatted_list = []
for item in table_of_contents_raw:
        title, pagenum = \
            split_to_title_and_pagenum(item) # data_func.split_to_title_and_pagenum(item)
        if title != None:
            title_list.append(title)
            pagenum_list.append(pagenum)
            title_formatted_list.append(
                convert_title_to_filename(title)) # data_func.convert_title_to_filename(title))
            
# for page_list, we need to add the last page as well
pagenum_list.append(num_of_pages + 1)

IndexError: ignored